[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1NF6pDYVgjrb02dsIEcZAW4JVsiUKPXFB#scrollTo=xIWQ1hWAu25g)


# Part.1 CRAWLING BERITA

## Rahma Nurhaliza (210411100176)

Teknik crawling adalah proses pengumpulan informasi dari berbagai sumber di internet secara otomatis. Web crawling biasanya dilakukan
oleh program yang disebut web crawler atau spider. Tujuan dari web crawling adalah untuk mengindeks konten di web sehingga dapat dianalisis.



## Install Library

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install tqdm

## Import Library

- request digunakan untuk meminta respon pada situs yang akan dilakukan crawling data
- BeatifulSoup digunakan untuk melakukan ekstraksi halaman konten agar dapat mengambil isi-isi konten pada berita di website
- pandas digunakan untuk manipulasi dan analisis data. Memungkinkan untuk bekerja secara tabuler
- tqdm digunakan untuk menampilkan progress bar saat menjalankan loop atau iterasi yang memerlukan waktu lama, sangat berguna ketika ingin memantau seberapa jauh proses iterasi telah berjalan,

In [ ]:
from os import replace
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

## CRAWLING BERITA CNN

Data berita berasal dari situs CNN


In [ ]:
def cnnnews(page, limit=10):  # Tambahkan parameter limit dengan default 10
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori': []}
    article_count = 0  # Menyimpan jumlah artikel yang sudah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break  # Hentikan loop jika jumlah artikel sudah mencapai limit

        url = f"https://www.cnnindonesia.com/ekonomi/indeks/5/{i}"
        r = requests.get(url)
        request = r.content
        soup = BeautifulSoup(request, 'html.parser')
        soup = soup.find('div', {'class': 'flex flex-col gap-5'})
        news = soup.findAll('article', {'class': 'flex-grow'})

        for new in tqdm(news):
            if article_count >= limit:
                break  # Menghentikan loop jika jumlah artikel sudah mencapai limit

            a_element = new.find('a', {'aria-label': 'link description'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = detail_soup.find('h1', {'class': 'leading-9'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil berita
            berita_div = detail_soup.find('div', {'class': 'detail-text'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Ambil tanggal
            tanggal = detail_soup.find('div', {'class': 'text-cnn_grey'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil kategori dari breadcrumb
            breadcrumb_items = detail_soup.find_all('a', {'class': 'text-cnn_black_light3'})
            kategori = breadcrumb_items[1].text.strip() if len(breadcrumb_items) > 1 else 'No category'

            # Bersihkan teks berita dari noise dan kata-kata yang tidak relevan
            noise = detail_soup.find('strong').text.strip() if detail_soup.find('strong') else ''
            berita = berita.replace("ADVERTISEMENT", "").replace("SCROLL TO CONTINUE WITH CONTENT", "").replace("Lihat Juga :", "").replace("Bagikan", "").replace(noise, "").replace("-- ", "").replace("--", "")
            berita = ' '.join(berita.split())  # Hilangkan spasi berlebih

            # Tambahkan data ke list
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1  # Penghitung artikel

    # Simpan data ke dalam CSV
    df = pd.DataFrame(data)
    df.to_csv("berita_cnn.csv", index=False)

    return df



In [ ]:
cnnnews(page=1, limit=10)

100%|██████████| 1/1 [00:10<00:00, 10.46s/it]


,judul,berita,tanggal,kategori
0,BI Catat Kinerja Penjualan Eceran Melambah Pad...,Bank Indonesia (BI) mencatat kinerja penjualan...,"Selasa, 10 Sep 2024 22:10 WIB",Ekonomi
1,Menhub Buka Peluang Gandeng Asing Kelola Banda...,Menteri Perhubungan Budi Karya Sumadi membuka ...,"Selasa, 10 Sep 2024 21:02 WIB",Ekonomi
2,Jokowi Groundbreaking 2 Investasi Asing di IKN...,Presiden Joko Widodo (Jokowi) dijadwalkan mela...,"Selasa, 10 Sep 2024 20:02 WIB",Ekonomi
3,Amran Copot Direktur Diduga Calo Pengadaan Bar...,Menteri Pertanian Andi Amran Sulaiman mencopot...,"Selasa, 10 Sep 2024 19:11 WIB",Ekonomi
4,Bahlil Bicara Pasokan Listrik di Tengah Rencan...,Menteri Energi dan Sumber Daya Mineral (ESDM) ...,"Selasa, 10 Sep 2024 18:32 WIB",Ekonomi
5,Pertamina Kaji Ubah Minyak Jelantah Jadi Avtur,PT Pertamina (Persero) mengkaji pengembangan b...,"Selasa, 10 Sep 2024 17:43 WIB",Ekonomi
6,"Dukung PMI, Bank Mandiri Perluas Akses Livin' ...",Bank Mandiri menghadirkan program Livin' Aroun...,"Selasa, 10 Sep 2024 17:18 WIB",Ekonomi
7,Pengamat Ingatkan Pertamina dan PLN Hati-hati ...,Pengamat Energi Komaidi Notonegoro meminta PT ...,"Selasa, 10 Sep 2024 17:12 WIB",Ekonomi
8,Transjakarta Perpanjang Jam Layanan Dua Rute S...,Transjakarta memperpanjang jam operasional lay...,"Selasa, 10 Sep 2024 16:46 WIB",Ekonomi
9,"IHSG Ditutup Menguat ke 7.761, Sektor Teknolog...",Indeks harga saham gabungan (IHSG) ditutup di ...,"Selasa, 10 Sep 2024 16:25 WIB",Ekonomi


## CRAWLING BERITA KOMPAS


Data berita berasal dari situs kompas, pada kategori SAINS



In [ ]:
def kompasnews(page, limit=15): #page: jumlh halaman yg ingin di crawl, limit: batas jumlah berita yang ingin diambil
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori':[]} #menyimpan informasi yang akan diambil
    article_count = 0 #menghitung berapa banyak artikel yang telah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break

        url = f"https://www.kompas.com/sains?source=navbar&page={i}" #(i) : nomor halaman saat ini, yaitu halaman yang akan dikunjungi crawler.
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Menemukan elemen artikel
        articles = soup.find_all('div', {'class': 'articleItem'})
        if not articles:
            print(f" menemukan 0 artikel pada halaman {i}")
            continue

        for article in articles:
            if article_count >= limit:
                break

            # Ambil URL artikel
            a_element = article.find('a', {'class': 'article-link'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = article.find('h2', {'class': 'articleTitle'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil tanggal
            tanggal = article.find('div', {'class': 'articlePost-date'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil tanggal
            breadcrumb_items = detail_soup.find_all('li', {'class': 'breadcrumb__item'})
            kategori = breadcrumb_items[1].get_text(strip=True) if breadcrumb_items else 'No category'


            # Ambil berita dari artikel detail
            berita_div = detail_soup.find('div', {'class': 'read__content'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Tambahkan ke data
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1

    df = pd.DataFrame(data)
    df.to_csv('berita-kompas-sains.csv', index=False)
    return df

In [ ]:
kompasnews(1,limit=10)

100%|██████████| 1/1 [00:12<00:00, 12.45s/it]


,judul,berita,tanggal,kategori
0,Para Ahli Memperingatkan Kasus Kanker Penis Me...,"KOMPAS.com - Kanker penis jarang terjadi, teta...",06/07/2024,Sains
1,"Berusia Lebih dari 50.000 Tahun, Lukisan Goa T...",KOMPAS.com - Tim peneliti yang terdiri dari Gr...,05/07/2024,Sains
2,Apakah Manusia Raksasa Pernah Ada di Bumi?,KOMPAS.com -- Manusia telah berbagi cerita ten...,05/07/2024,Sains
3,"Kisah Tragis Penelope, Hiu Hamil yang Dimangsa...",KOMPAS.com - Hiu sering dikenal sebagai predat...,05/09/2024,Sains
4,Menggagas Satelit Konstelasi Pemantau Bumi Nus...,Oleh: Wahyudi Hasbi*\nINDONESIA dengan lebih d...,31/08/2024,Sains
5,Menjaga Sinergi untuk Ketahanan Pangan Nasional,Oleh: Rubiyo dan Dwinita Wikan Utami\nINDONESI...,25/08/2024,Sains
6,Bagaimana Adik Bayi Dibuat?,"Bagaimana bayi dibuat?\n\t\t\t\t\t\t- Giaan, u...",24/08/2024,Sains
7,Kisah Pencurian Mona Lisa 113 Tahun Lalu yang ...,KOMPAS.com - 21 Agustus 1911 adalah salah satu...,21/08/2024,Sains
8,5 Manfaat Campuran Bawang Putih dan Jahe untuk...,KOMPAS.com - Bawang putih dan jahe sama-sama t...,29/01/2024,Sains
9,Danau Singkarak Sumatera Barat Diguncang 5 Ren...,"KOMPAS.com - Area Danau Singkarak, Sumatera Ba...",28/01/2024,Sains


Data dari situs kompas kategori Edukasi

In [ ]:
def kompasnews(page, limit=15): #page: jumlh halaman yg ingin di crawl, limit: batas jumlah berita yang ingin diambil
    data = {'judul': [], 'berita': [], 'tanggal': [], 'kategori':[]} #menyimpan informasi yang akan diambil
    article_count = 0 #menghitung berapa banyak artikel yang telah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break

        url = f"https://edukasi.kompas.com/?source=navbar&page={i}" #(i) : nomor halaman saat ini, yaitu halaman yang akan dikunjungi crawler.
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Menemukan elemen artikel
        articles = soup.find_all('div', {'class': 'articleItem'})
        if not articles:
            print(f" menemukan 0 artikel pada halaman {i}")
            continue

        for article in articles:
            if article_count >= limit:
                break

            # Ambil URL artikel
            a_element = article.find('a', {'class': 'article-link'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = article.find('h2', {'class': 'articleTitle'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil tanggal
            tanggal = article.find('div', {'class': 'articlePost-date'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil tanggal
            breadcrumb_items = detail_soup.find_all('li', {'class': 'breadcrumb__item'})
            kategori = breadcrumb_items[1].get_text(strip=True) if breadcrumb_items else 'No category'


            # Ambil berita dari artikel detail
            berita_div = detail_soup.find('div', {'class': 'read__content'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Tambahkan ke data
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)
            data["kategori"].append(kategori)

            article_count += 1

    df = pd.DataFrame(data)
    df.to_csv('berita-kompas-Edukasi.csv', index=False)
    return df

In [ ]:
kompasnews(1,limit=10)

100%|██████████| 1/1 [00:08<00:00,  8.78s/it]


,judul,berita,tanggal,kategori
0,"Cerita Kadit CoC, Pilih Kuliah Teknik Elektro ...",KOMPAS.com - Kaditya Rakan adalah juara ketiga...,10/09/2024,Edu
1,Beasiswa Ikatan Dinas PT IMIP bagi Mahasiswa D...,KOMPAS.com - Mahasiswa UGM (Universitas Gadjah...,10/09/2024,Edu
2,"Gelar Beasiswa ""ScholarFlip"", Flip Ajak Mahasi...","KOMPAS.com – Flip, salah satu fintech Indonesi...",10/09/2024,Edu
3,Dosen Unesa Ungkap Alasan Pemberian ASI Harus ...,"KOMPAS.com - Setelah lahir, bayi sangat direko...",10/09/2024,Edu
4,Dukungan dan Investasi Dibutuhkan Generasi Mud...,KOMPAS.com - Populasi generasi muda diprediksi...,10/09/2024,Edu
5,Kemendikbud Targetkan 10.000 Wirausaha Baru le...,"KOMPAS.com - Kementerian Pendidikan, Kebudayaa...",10/09/2024,Edu
6,Dukungan dan Investasi Dibutuhkan Generasi Mud...,KOMPAS.com - Populasi generasi muda diprediksi...,10/09/2024,Edu
7,Jusuf Kalla: Indonesia Masih Perlu Adakan Ujia...,KOMPAS.com - Wakil Presiden ke-10 dan ke-12 Ju...,10/09/2024,Edu
8,"Asesmen Nasional Jenjang SMP 2024 Dimulai, Ada...",KOMPAS.com - Asesmen Nasional Berbasis Kompute...,10/09/2024,Edu
9,"Amygdala: Pengertian, Fungsi, dan Anatominya",KOMPAS.com - Amygdala adalah tempat disimpanny...,10/09/2024,Skola


Menggabungkan file berita dari kategori yang berbeda

In [ ]:
df_ikn = pd.read_csv('berita-kompas-sains.csv')
df_edukasi = pd.read_csv('berita-kompas-Edukasi.csv')


# Menggabungkan dua DataFrame
df_combined = pd.concat([df_edukasi, df_ikn])

# Menyimpan hasil gabungan ke file baru
df_combined.to_csv('berita-kompas-combined.csv', index=False)
df_combined


,judul,berita,tanggal,kategori
0,"Cerita Kadit CoC, Pilih Kuliah Teknik Elektro ...",KOMPAS.com - Kaditya Rakan adalah juara ketiga...,10/09/2024,Edu
1,Beasiswa Ikatan Dinas PT IMIP bagi Mahasiswa D...,KOMPAS.com - Mahasiswa UGM (Universitas Gadjah...,10/09/2024,Edu
2,"Gelar Beasiswa ""ScholarFlip"", Flip Ajak Mahasi...","KOMPAS.com – Flip, salah satu fintech Indonesi...",10/09/2024,Edu
3,Dosen Unesa Ungkap Alasan Pemberian ASI Harus ...,"KOMPAS.com - Setelah lahir, bayi sangat direko...",10/09/2024,Edu
4,Dukungan dan Investasi Dibutuhkan Generasi Mud...,KOMPAS.com - Populasi generasi muda diprediksi...,10/09/2024,Edu
5,Kemendikbud Targetkan 10.000 Wirausaha Baru le...,"KOMPAS.com - Kementerian Pendidikan, Kebudayaa...",10/09/2024,Edu
6,Dukungan dan Investasi Dibutuhkan Generasi Mud...,KOMPAS.com - Populasi generasi muda diprediksi...,10/09/2024,Edu
7,Jusuf Kalla: Indonesia Masih Perlu Adakan Ujia...,KOMPAS.com - Wakil Presiden ke-10 dan ke-12 Ju...,10/09/2024,Edu
8,"Asesmen Nasional Jenjang SMP 2024 Dimulai, Ada...",KOMPAS.com - Asesmen Nasional Berbasis Kompute...,10/09/2024,Edu
9,"Amygdala: Pengertian, Fungsi, dan Anatominya",KOMPAS.com - Amygdala adalah tempat disimpanny...,10/09/2024,Skola
